In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cluster import KMeans


In [2]:
from google.colab import files
uploaded = files.upload()

Saving Text_Clustering.xlsx to Text_Clustering.xlsx


In [6]:
data=pd.read_excel('Text_Clustering.xlsx') #reading the file and store in pandas dataframe
data.head() # to check the file is correctly read

,text,subject
0,The idea is to create a ticketing support syst...,NaN
1,Level 1 - Integrate Machine Learning Algorit...,NaN
2,\nCustomers could be well informed in advance ...,NaN
3,Have a high level categorization of the histor...,NaN
4,Get information from customer on the systems...,NaN


In [7]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
STOPWORDS = set(stopwords.words("english")) # you may add new stop words here 
print(STOPWORDS) # check the stop words 

{'against', 'all', 'shouldn', 'themselves', 'your', 'who', 'has', 'until', 'during', 'under', 'again', 'won', 'these', 'be', 'can', 'theirs', "mightn't", 'an', 'up', "doesn't", 'was', 'don', 'from', 're', 'herself', "hadn't", 'down', 'his', 'himself', 'y', "you've", 'about', "shan't", 'ma', 'before', 'both', 'than', 'o', 'then', 'that', 'whom', 'why', 'to', 'in', 't', 'which', 'ain', "weren't", 'm', "you'd", 'over', 'our', 'after', 'own', 'off', 'no', 'haven', "couldn't", 'by', "won't", 'as', 'will', "should've", 'are', 'yourself', 'couldn', "it's", 's', 'yourselves', 'now', 'doing', 'do', 'for', 'some', 'itself', 'shan', 'once', 'other', 'mightn', 'between', 'been', 'hasn', 'i', 'should', 'wasn', "that'll", 'weren', 'were', 'such', "wouldn't", 'or', 'same', 'hers', 'them', 'so', 'aren', 'because', "isn't", "wasn't", 'very', 'me', 'having', "didn't", "haven't", 'she', 'here', 'of', 'her', 'on', "mustn't", "you'll", 'they', 'needn', 'too', 'but', 'is', 'nor', 'what', 'into', 'ours', "ha

In [9]:
#Regular expression for removing some pattenrs from the text
import re
import string
def clean_text(text, tokenizer, stopwords):
    text = str(text).lower()  # Lowercase words
    text = re.sub(r"\[(.*?)\]", "", text)  # Remove  the chars in content
    text = re.sub(r"\s+", " ", text)  # Remove multiple spaces in content
    text = re.sub(r"\w+…|…", "", text)  # Remove alpha numeric cahrecters (and last word)
    text = re.sub(r"(?<=\w)-(?=\w)", " ", text)  # Replace dash between words
    text = re.sub(
        f"[{re.escape(string.punctuation)}]", "", text
    )  # Remove punctuation

    tokens = tokenizer(text)  # Get tokens from text
    tokens = [t for t in tokens if not t in stopwords]  # Remove stopwords
    tokens = ["" if t.isdigit() else t for t in tokens]  # Remove digits
    tokens = [t for t in tokens if len(t) > 1]  # Remove short tokens
    return tokens

In [10]:
from nltk import word_tokenize
nltk.download('punkt')
#create a new column "tokens" and store the tokens 
data["tokens"] = data['text'].map(lambda x: clean_text(x, word_tokenize, STOPWORDS))
data["tokens"].head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


0    [idea, create, ticketing, support, system, whe...
1    [level, integrate, machine, learning, algorith...
2    [customers, could, well, informed, advance, pr...
3    [high, level, categorization, historical, issu...
4    [get, information, customer, systems, shows, s...
Name: tokens, dtype: object

In [11]:
from nltk.stem import PorterStemmer

In [12]:
ps = PorterStemmer() #stemmer object 
def stemSentence(token_words):
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(ps.stem(word))
    return (stem_sentence)

In [13]:
#convert all the tokens to its stem
data["tokens"] = data["tokens"].map(lambda x: stemSentence(x))
data.head()

,text,subject,tokens
0,The idea is to create a ticketing support syst...,NaN,"[idea, creat, ticket, support, system, wherein..."
1,Level 1 - Integrate Machine Learning Algorit...,NaN,"[level, integr, machin, learn, algorithm, use,..."
2,\nCustomers could be well informed in advance ...,NaN,"[custom, could, well, inform, advanc, probabl,..."
3,Have a high level categorization of the histor...,NaN,"[high, level, categor, histor, issu, identifi,..."
4,Get information from customer on the systems...,NaN,"[get, inform, custom, system, show, spike, ope..."


In [14]:
#Now convert the cleaned text to TF-IDF vector,only for tokens column
tokens_raw=[" ".join(t) for t in data.tokens]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(tokens_raw)
print(X)

  (0, 444)	0.13755630845212422
  (0, 1315)	0.13755630845212422
  (0, 216)	0.09373170419748046
  (0, 101)	0.12132761652456978
  (0, 1490)	0.07192108225544298
  (0, 1038)	0.12132761652456978
  (0, 107)	0.059124825292490336
  (0, 2358)	0.12132761652456978
  (0, 1453)	0.1050989245970153
  (0, 90)	0.09199661029074771
  (0, 208)	0.08250343407737222
  (0, 1249)	0.057536891225126056
  (0, 2162)	0.13755630845212422
  (0, 280)	0.10234126409781878
  (0, 1781)	0.08141425846881847
  (0, 2025)	0.08887023266946086
  (0, 2750)	0.1607470139592476
  (0, 47)	0.18746340839496092
  (0, 1652)	0.11610314464981826
  (0, 1324)	0.06364128912157382
  (0, 237)	0.04869231752355978
  (0, 195)	0.2561262644774975
  (0, 1154)	0.11610314464981826
  (0, 2085)	0.08250343407737222
  (0, 777)	0.19974890544452759
  :	:
  (257, 302)	0.3881312200702327
  (257, 588)	0.3881312200702327
  (257, 971)	0.32759860811915226
  (257, 2001)	0.29654891361166796
  (257, 2770)	0.2755110099642703
  (257, 1511)	0.2697628061077614
  (257, 147

In [15]:
from sklearn.cluster import KMeans
nc = 7 # Number of Clusters
km = KMeans(n_clusters=nc, init = 'k-means++',max_iter = 100, n_init = 10)
km.fit(X)

KMeans(max_iter=100, n_clusters=7)

In [16]:
sorted_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(nc):
        print("Cluster %d:" % i, end='')
        for ind in sorted_centroids[i, :10]:
            print(' %s' % terms[ind], end='')
        print()
        print()
print()

Cluster 0: data db mongodb db2 mongo chang system databas busi applic

Cluster 1: ticket issu team support resolut system incid time problem resolv

Cluster 2: user app base use data locat system like solut inform

Cluster 3: flood item drive good landmark map insid gi data plant

Cluster 4: virtual intellig nan artif assist payment content analysi person financi

Cluster 5: product mar custom food recip mobil inform app use user

Cluster 6: credit card blockchain loan score transact use fraud solut ai




/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [17]:
test_data = ["You should have told me yesterday"]
Y = vectorizer.transform(test_data)
prediction = km.predict(Y)
print(prediction)

[2]
